In [2]:
import numpy as np, pandas as pd
path = 'C:/Users/Glenn Wright/Contacts/Documents/GitHub/homework/'
base = pd.read_csv(path+'meps_base_data.csv', sep=',')
meds = pd.read_csv(path+'meps_meds.csv', sep=',')